In [615]:
import torch
import os
import cv2
import numpy as np
from functools import cmp_to_key

In [616]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained = True)

Using cache found in /Users/victorzarzu/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2023-4-16 Python-3.10.6 torch-2.0.0 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


requirements: /Users/victorzarzu/.cache/torch/hub/requirements.txt not found, check failed.


In [617]:
BASE_URL = '/Users/victorzarzu/ubb/year2/sem2/ai/lab/lab7/coco128/'
images_dir = BASE_URL + 'images/train2017' 
labels_dir = BASE_URL + 'labels/train2017' 

In [618]:
def readLabelFile(filename):
    result = []
    for line in open(filename, 'r'):
        row = []
        for value in line.split(' '):
            row.append(float(value))
        result.append(row)
    return result

In [619]:
def readLabels(filenames): 
    result = []
    for filename in filenames:
        result.append(readLabelFile(filename))
    return result

In [620]:
def readData():
    images = []
    labelsFiles = []
    for info in os.listdir(images_dir):
        images.append(os.path.join(images_dir, info))
        labelsFiles.append(os.path.join(labels_dir, info.replace('jpg', 'txt')))

        #results = model([os.path.join(images_dirpath, image) for image in images])
        #print(results.show())
    #print(labelsFiles)
    labels = readLabels(labelsFiles)
    return [images, labels]

[images, labels] = readData()

In [621]:
def compare(x, y):
    for valx, valy in zip(x[1:], y[1:]):
        if valx < valy:
            return -1
        elif valx > valy:
            return 1
    return 0

In [622]:
class YOLOLoss:

    def __init__(self, images, predictions, labels):
        self.__images = images
        self.__predictions = predictions
        self.__labels = labels
    
    def __toExtremities(self, labels, width, height):
        for label in labels:
            for i in range(1, len(label)):
                if i % 2 == 1:
                    label[i] *= width
                else:
                    label[i] *= height
            
            bbWidth = label[3]
            bbHeight = label[4]
            label[1] -= bbWidth / 2
            label[2] -= bbHeight / 2
            label[3] = label[1] + bbWidth
            label[4] = label[2] + bbHeight
        
        return labels

    def __transformPredictions(self, predictions):
        result = []
        for i in range(len(predictions)):
            result.append([predictions[i][-1]])
            result[i].extend(list(predictions[i][:4]))
        return result

    def __singleIoU(self, predictions, labels):
        x_top_p, y_top_p, x_bottom_p, y_bottom_p = predictions[1], predictions[2], predictions[3], predictions[4]
        x_top_l, y_top_l, x_bottom_l, y_bottom_l = labels[1], labels[2], labels[3], labels[4]

        x_top = max(x_top_p, x_top_l)
        y_top = max(y_top_p, y_top_l)
        x_bottom = min(x_bottom_p, x_bottom_l)
        y_bottom = min(y_bottom_p, y_bottom_l)

        intersection_area = (x_bottom - x_top + 1) * (y_bottom - y_top + 1)

        if x_top >= x_bottom or y_top >= y_bottom:
            intersection_area = 0
        
        predicted_area = (x_bottom_p - x_top_p + 1) * (y_bottom_p - y_top_p + 1)
        label_area = (x_bottom_l - x_top_l + 1) * (y_bottom_l - y_top_l + 1)

        unioun_area = (predicted_area + label_area - intersection_area)

        return intersection_area / unioun_area

    def __IoU(self, predictions, labels):
        predictions = sorted(predictions, key = cmp_to_key(compare))
        labels = sorted(labels, key = cmp_to_key(compare))

        iou = 0

        for prediction, label in zip(predictions, labels):
            if prediction[0] == label[0]:
                iou += self.__singleIoU(prediction, label)
        
        return iou

    def imageError(self, image, predictions, labels):
        img = cv2.imread(image, cv2.IMREAD_UNCHANGED)
        height, width = img.shape[0], img.shape[1]

        labels = self.__toExtremities(labels, width, height)
        predictions = self.__transformPredictions(predictions)
        return self.__IoU(predictions, labels)
    
    def imagesError(self, images, predicitons, labels):
        error = 0 
        for image, prediction, label in zip(images, predicitons, labels):
            error += self.imageError(image, prediction, label)
        
        return error

In [623]:
results = model(images[:10])
loss = YOLOLoss(images = images, predictions = results.xyxy, labels = labels)
error = loss.imagesError(images, results.xyxy, labels)
print(error)

tensor(20.11018)
